In [1]:
import control as ct
import numpy as np

In [28]:
g = 9.8
mass = 0.027
l = 0.0397
Ixx=1.4e-5
Iyy=1.4e-5 
Izz=2.17e-5

phi=0
theta=0
psi=0
u=0
v=0
w=0
p=0
q=0
r=0
        

A = np.array([[0, 0, 0, v*theta+w*psi, v*phi+w, -v+w*phi, 1, theta*phi-psi, theta+psi*phi, 0, 0, 0], #x
                [0, 0, 0, v*psi*theta-w, v*psi*phi+w*psi, u+v*theta*phi+w*theta, psi, psi*theta*phi+1, psi*theta-phi, 0, 0, 0], #y
                [0, 0, 0, v, -u, 0, -theta, phi, 1, 0, 0, 0], #z
                [0, 0, 0, q*theta, q*phi+r, 0, 0, 0, 0, 1, phi*theta, theta], #phi
                [0, 0, 0, -r, 0, 0, 0, 0, 0, 0, 1, -phi], #theta
                [0, 0, 0, q, 0, 0, 0, 0, 0, 0, phi, 1], #psi
                [0, 0, 0, 0, g, 0, 0, r, -q, 0, -w, v], #u
                [0, 0, 0, -g, 0, 0, -r, 0, p, w, 0, -u], #v
                [0, 0, 0, 0, 0, 0, q, -p, 0, -v, u, 0], #w
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, r*(Iyy-Izz)/Ixx, q*(Iyy-Izz)/Ixx], #p
                [0, 0, 0, 0, 0, 0, 0, 0, 0, r*(Izz-Ixx)/Iyy, 0, p*(Izz-Ixx)/Iyy], #q
                [0, 0, 0, 0, 0, 0, 0, 0, 0, q*(Ixx-Iyy)/Izz, p*(Ixx-Iyy)/Izz, 0]]) #r
        
        
B = np.array([[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [1/mass, 0, 0, 0],
                [0, 1/Ixx, 0, 0],
                [0, 0, 1/Iyy, 0],
                [0, 0, 0, 1/Izz]])

C = np.eye(12)
D = np.zeros((12,4))

quadrotor_model = ct.ss(A, B, C, D)

In [56]:
Q = np.diag([1000, 1000, 1000, 10, 10, 100, 50, 50, 100, 0.1, 0.1, 0.1])
#x, y, z, phi, theta, psi, u, v, w, p, q, r
R = np.diag([10, 10, 10, 10])
#Thrust, Mx, My, Mz

K, S, E = ct.lqr(quadrotor_model, Q, R)
K[K < 1e-6] = 0
K[3]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       3.16227766, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.10068388])

In [ ]:
K = [[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
     [0, -1, 0, 1, 0, 0, 0, -1, 0, 1, 0, 0], #Mx roll
     [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0], #My pitch
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]] 

In [49]:
E

array([-7.1428501e+03+0.j      , -1.1291620e+01+9.373105j,
       -1.1291620e+01-9.373105j, -4.5506139e+00+0.j      ,
       -1.1707867e+02+0.j      , -3.1634319e+00+0.j      ,
       -7.1428501e+03+0.j      , -4.6081865e+03+0.j      ,
       -1.1291619e+01+9.373105j, -1.1291619e+01-9.373105j,
       -4.5506139e+00+0.j      , -3.1623522e+01+0.j      ],
      dtype=complex64)

## Controllability

In [9]:
C_Matrix = ct.ctrb(A, B)
C_rank = np.linalg.matrix_rank(C_Matrix)
print(f"rank: {C_rank}")
# Controlability Gramian

rank: 12


## Observability

In [13]:
O_Matrix = ct.obsv(A, C)
O_rank = np.linalg.matrix_rank(O_Matrix)
print(f"rank: {O_rank}")

rank: 12


## Controller Design

In [ ]:
K, S, E = ct.lqr(quad_linear_model, Q, R, N)

## Trajectory Analysis

## Multi-Agent System Analysis

In [ ]:
ct.lyap()

In [6]:
import numpy as np
phi = 0

g = 9.8
m = 0.27
Ixx = 2e-5
Iyy = 2e-5
Izz = 5e-5
l = 1
Ka = 1e-9
Km = 1e-7

a_sub = np.array([[0, 0, 0, g*np.sin(phi), g*np.cos(phi), 0],
                    [0, 0, 0, -g*np.cos(phi), g*np.sin(phi), 0]])
a_sub = np.vstack((a_sub, np.zeros((4, 6))))


A = np.block([[np.zeros((6,6)), np.eye(6)],
                [a_sub, np.zeros((6,6))]])

b_sub = np.array([[Ka/m, Ka/m, Ka/m, Ka/m],
                    [0, -Ka*l/Ixx, 0, Ka*l/Ixx],
                    [Ka*l/Iyy, 0, -Ka*l/Iyy, 0],
                    [Km/Izz, -Km/Izz, Km/Izz, -Km/Izz]])

B = np.vstack((np.zeros((8, 4)), b_sub))

C = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])


In [7]:
A.shape, B.shape

((12, 12), (12, 4))

## Lyapunov Stability